In [1]:
import pandas as pd

In [2]:
movie_train=pd.read_csv("movie_review_train.csv")
movie_test=pd.read_csv("movie_review_test.csv")

In [3]:
movie_train["class"].unique()

array(['Pos', 'Neg'], dtype=object)

In [4]:
movie_train["class"]=movie_train["class"].map({"Pos":1,"Neg":0})
movie_test["class"]=movie_test["class"].map({"Pos":1,"Neg":0})

In [5]:
movie_train["class"].value_counts()

1    800
0    800
Name: class, dtype: int64

In [6]:
movie_test["class"].value_counts()

1    200
0    200
Name: class, dtype: int64

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words='english')

In [8]:
X_train=movie_train["text"]
y_train=movie_train["class"]

X_test=movie_test["text"]
y_test=movie_test["class"]


In [9]:
di=vect.fit(X_train)
temp={}
X_train_vocabs_dict = di.get_feature_names()
for i in X_train_vocabs_dict:
    if temp.get(i):
        temp[i]=temp[i]+1
    else:
        temp[i]=1


In [10]:
word=[]
coun=[]
for i in vect.vocabulary_:
    coun.append(vect.vocabulary_[i])
    word.append(i)
#     temp.append()

In [11]:
df=pd.DataFrame({"word":word,"count":coun})

In [12]:
df["percentage"]=df["count"].apply(lambda s: round((float(s)/35858),1))

In [13]:
final_df=df[(df.percentage>=0.03) & (df.percentage<=0.8)]
len(final_df)

28687

In [14]:
vect3 = CountVectorizer(stop_words='english',min_df=.03,max_df=.8)
vect3.fit(X_train)
X_train_vocabs = vect3.get_feature_names()
len(X_train_vocabs)


1643

In [15]:
vect3 = CountVectorizer(stop_words='english',min_df=.03,max_df=.8)
vect3.fit(X_test)

X_train_transformed = vect3.transform(X_train)
X_train_transformed.count_nonzero()


212938

In [16]:
vect3 = CountVectorizer(stop_words='english',min_df=.03,max_df=.8)
vect3.fit(X_train)
# transforming the train and test datasets
X_train_transformed = vect3.transform(X_train)
X_test_transformed =vect3.transform(X_test)



In [17]:
# training the NB model and making predictions
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

# fit
mnb.fit(X_train_transformed,y_train)

# predict class
y_pred_class = mnb.predict(X_test_transformed)

# predict probabilities
y_pred_proba =mnb.predict_proba(X_test_transformed)


# printing the overall accuracy
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)


0.8275

In [18]:
mnb

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
# confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[172,  28],
       [ 41, 159]])

In [20]:
confusion = metrics.confusion_matrix(y_test, y_pred_class)
print(confusion)
#[row, column]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
TP = confusion[1, 1]
FP

[[172  28]
 [ 41 159]]


28

In [21]:
word_count=mnb.feature_count_.sum(axis=1)
pwc=word_count[1]
pwc

153000.0

In [22]:
sum(list(mnb.feature_count_[0]))


137807.0

In [23]:
vect3.vocabulary_.get("stupid")

1390

In [24]:
Neg_token_count = mnb.feature_count_[0, :]
Pos_token_count = mnb.feature_count_[1, :]
tokens = pd.DataFrame({'words':X_train_vocabs, 'Pos':Pos_token_count, 'Neg':Neg_token_count}).set_index('words')
tokens.loc['stupid', ]

Neg    161.0
Pos     35.0
Name: stupid, dtype: float64

In [25]:

Neg_token_count = mnb.feature_count_[0, :]
Pos_token_count = mnb.feature_count_[1, :]
tokens = pd.DataFrame({'words':X_train_vocabs, 'Pos':Pos_token_count, 'Neg':Neg_token_count}).set_index('words')
tokens.loc['painfully', ]



Neg    41.0
Pos    10.0
Name: painfully, dtype: float64

In [26]:
sum(list(mnb.feature_count_[1]))


153000.0

In [27]:
tokens["N"]=tokens["Neg"].apply(lambda s:s/sum(list(mnb.feature_count_[0])))
tokens["P"]=tokens["Pos"].apply(lambda s:s/sum(list(mnb.feature_count_[1])))

In [30]:
tokens["r"]=tokens["N"]/tokens["P"]

In [35]:
tokens.sort_values("r")

,Neg,Pos,N,P,r
words,,,,,
outstanding,5.0,57.0,0.000036,0.000373,0.097390
wonderfully,9.0,59.0,0.000065,0.000386,0.169360
pulp,13.0,74.0,0.000094,0.000484,0.195044
cameron,21.0,119.0,0.000152,0.000778,0.195926
era,14.0,67.0,0.000102,0.000438,0.231992
satisfying,10.0,46.0,0.000073,0.000301,0.241358
fantastic,10.0,45.0,0.000073,0.000294,0.246722
superb,14.0,62.0,0.000102,0.000405,0.250701
allows,14.0,59.0,0.000102,0.000386,0.263449
